In [1]:
include("../code/exact_enum.jl")
include("../code/matrix_generator.jl")
include("../code/bp_full.jl")

fix_indep! (generic function with 1 method)

In [4]:
r = 0.2
N = 100
M = round(Int, N*(1-r))
f = 1-3r
AA = permutedims(ldpc_matrix(N,M,3M,[0,1-f,f],[0, 0, 1]; accept_multi_edges=false))
src = bitrand(N)
BB, ind = findbasis_slow(BitMatrix(AA));

In [8]:
@time exact_wef(BB, [src]);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:04


  4.887247 seconds (906 allocations: 69.297 KiB)


In [2]:
# plot_wef(h[1], size=(400,200))

## Vary the rate

In [9]:
include("../code/exact_enum.jl")
include("../code/matrix_generator.jl")
include("../code/bp_full.jl")
include("../../telegram/notifications.jl")

default_tg_text (generic function with 1 method)

In [16]:
Rs = 0.04:0.04:0.34
ns = [25, 50, 75]
navg = [500, 200, 50]
d = [[fill(NaN, navg[i]) for _ in Rs] for i in eachindex(ns)]
d_ms = [[fill(NaN, navg[i]) for _ in Rs] for i in eachindex(ns)]
nz = [[fill(NaN, navg[i]) for _ in Rs] for i in eachindex(ns)]
wef_src = [[[zeros(Int, ns[i]) for _ in 1:navg[i]] for _ in Rs] for i in eachindex(ns)]
wef_zero = [[[zeros(Int, ns[i]) for _ in 1:navg[i]] for _ in Rs] for i in eachindex(ns)]
t = fill(NaN, length(ns));

In [ ]:
@telegram "Exact enumeration" for (a,n) in enumerate(ns)
    println("### Size $a of ", length(ns))
    tt = @timed for (i,R) in enumerate(Rs)
        println("Rate $i of ", length(Rs))
        m = round(Int, n*(1-R))
        f3 = 1-3R
        Λ = [0,1-f3,f3]
        K = [0, 0, 1]
        nedges = 3m
        for j in 1:navg[a]       
            H = permutedims(ldpc_matrix(n,m,nedges,Λ,K; accept_multi_edges=false))
            s = bitrand(n)
            B, indep = findbasis_slow(BitMatrix(H))
            nz[a][i][j] = sum(!iszero, B) / prod(size(B))
            h0, h, mins = exact_wef(B, [s])
            wef_zero[a][i][j] = h0
            wef_src[a][i][j] = h[1]
            d[a][i][j] = mins[1]
            # maxsum
            efield = [(float(!ss), float(ss)) .+ 1e-5.*(randn(),randn()) for ss in s]
            ms = BPFull(H, efield)
            iteration_ms!(ms, maxiter=10^3, rein=1e-3)
            x = argmax.(ms.belief) .== 2
            p = parity(H, x)
            p != 0 && fix_indep!(x, B, indep)
            d_ms[a][i][j] = distortion(x, s) / n
        end
    end
    t[a] = tt.time
    d
end

### Size 1 of 3
Rate 1 of 8
Rate 2 of 8
Rate 3 of 8
Rate 4 of 8
Rate 5 of 8
Rate 6 of 8
Rate 7 of 8
Rate 8 of 8
### Size 2 of 3
Rate 1 of 8
Rate 2 of 8
Rate 3 of 8
Rate 4 of 8
Rate 5 of 8
Rate 6 of 8
Rate 7 of 8
Rate 8 of 8
### Size 3 of 3
Rate 1 of 8
Rate 2 of 8
Rate 3 of 8
Rate 4 of 8
Rate 5 of 8
Rate 6 of 8
Rate 7 of 8


Progress: 100%|█████████████████████████████████████████| Time: 0:00:07
Progress: 100%|█████████████████████████████████████████| Time: 0:00:07
Progress: 100%|█████████████████████████████████████████| Time: 0:00:07
Progress: 100%|█████████████████████████████████████████| Time: 0:00:07
Progress: 100%|█████████████████████████████████████████| Time: 0:00:07
Progress: 100%|█████████████████████████████████████████| Time: 0:00:07
Progress: 100%|█████████████████████████████████████████| Time: 0:00:07
Progress: 100%|█████████████████████████████████████████| Time: 0:00:07
Progress: 100%|█████████████████████████████████████████| Time: 0:00:07
Progress: 100%|█████████████████████████████████████████| Time: 0:00:07
Progress: 100%|█████████████████████████████████████████| Time: 0:00:07
Progress: 100%|█████████████████████████████████████████| Time: 0:00:07
Progress: 100%|█████████████████████████████████████████| Time: 0:00:07
Progress: 100%|█████████████████████████████████████████| Time: 

Rate 8 of 8


Progress: 100%|█████████████████████████████████████████| Time: 0:01:06
Progress: 100%|█████████████████████████████████████████| Time: 0:01:05
Progress: 100%|█████████████████████████████████████████| Time: 0:01:08
Progress: 100%|█████████████████████████████████████████| Time: 0:01:12
Progress: 100%|█████████████████████████████████████████| Time: 0:01:13
Progress: 100%|█████████████████████████████████████████| Time: 0:01:11
Progress: 100%|█████████████████████████████████████████| Time: 0:01:04
Progress: 100%|█████████████████████████████████████████| Time: 0:01:04
Progress: 100%|█████████████████████████████████████████| Time: 0:01:05
Progress: 100%|█████████████████████████████████████████| Time: 0:01:05
Progress: 100%|█████████████████████████████████████████| Time: 0:01:05
Progress: 100%|█████████████████████████████████████████| Time: 0:01:06
Progress: 100%|█████████████████████████████████████████| Time: 0:01:06
Progress:  52%|█████████████████████▍                   |  ETA: 

In [ ]:
d_avg = [mean.(d[i]) for i in eachindex(ns)]
d_std = [std.(d[i]) ./ sqrt(navg[i]) for i in eachindex(ns)];

In [ ]:
t_minutes = sum(t) / 60
println("Runtime: ", round(t_minutes,digits=1), " minutes")
pl = plot_rdb(f3=true)
for (i,n) in enumerate(ns)
    plot!(pl, Rs, d_avg[i], yerr=d_std[i], ms=3, label="n=$n, avg over $(navg[i])", marker=:o)
end
plot!(pl, title="Exact enumeration")

In [ ]:
d_ms_avg = [mean.(d_ms[i]) for i in eachindex(ns)]
d_ms_std = [std.(d_ms[i]) ./ sqrt(navg[i]) for i in eachindex(ns)];
for (i,n) in enumerate(ns)
    plot!(pl, Rs, d_ms_avg[i], yerr=d_ms_std[i], ms=3, label="maxsum n=$n, avg over $(navg[i])", marker=:o)
end
pl

In [ ]:
plot_wef(wef_zero[end][end][1])